In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import os
def change_dolar(local_value:float,
    change_rate:float):
  return local_value*change_rate




path_change=r"/content/drive/MyDrive/Hack Corruption - Contractor/datos/Tasa_cambio.xlsx"
path_data=r"/content/drive/MyDrive/Hack Corruption - Contractor/datos/records.csv"
path_CPI=r"/content/drive/MyDrive/Hack Corruption - Contractor/datos/USA_CPI.xlsx"
path_data_gener=r"/content/drive/MyDrive/Hack Corruption - Contractor/datos/zips/"

n=0

for path in os.listdir(path_data_gener):


  #Loading the exchange rate for USD standaricing
  exchange_rate=pd.read_excel(path_change)
  exchange_rate["exchange_rate"]=exchange_rate.apply(lambda row:(row["Máximo"]+row["Mínimo"])/2,
                                            axis=1)
  exchange_rate=exchange_rate[["Fecha","exchange_rate"]]

  #Loading the dataset  and cleaning dates
  records=pd.read_csv(path_data_gener+path)
  records=records.dropna(subset="compiledRelease/tender/bidOpening/date")


  records["Fecha"]=records["compiledRelease/tender/bidOpening/date"].apply(
      lambda x:dt.datetime.strptime(x[:10],"%Y-%m-%d").year
  )

  #Loading CPI for real value updating
  CPI=pd.read_excel(path_CPI)
  CPI=CPI[["Year","Avg"]].dropna()
  last_year=CPI["Avg"][len(CPI)-1]

  #merging tables
  records=records.merge(exchange_rate,how="left",on=["Fecha"])

  #We use the last inflation data and normalize everything to the last year
  records=records.merge(CPI,how="left",left_on=["Fecha"],right_on=["Year"])
  records["Avg"]=records["Avg"].fillna(last_year).apply(lambda x:x/last_year)

  #We scale every value in million dolars
  records["value_million_dolar"]=records.apply(lambda row:row["compiledRelease/tender/value/amount"]/(row["exchange_rate"]*row["Avg"]*1e3)
                                      if row["compiledRelease/tender/value/currency"]=="PYG" else row["compiledRelease/tender/value/amount"]/(row["Avg"]*1e3),axis=1)

  if n==0:
    joined=records
  else:
    joined=joined.append(records)


joined[["compiledRelease/planning/budget/description","value_million_dolar"]].dropna(subset=["compiledRelease/planning/budget/description","value_million_dolar"]).to_csv(r"/content/drive/MyDrive/Hack Corruption - Contractor/datos/limpio/value.csv",index=False)


<ipython-input-1-3ce846fd6354>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  records["Fecha"]=records["compiledRelease/tender/bidOpening/date"].apply(
<ipython-input-1-3ce846fd6354>:29: DtypeWarning: Columns (60,61,62,63) have mixed types. Specify dtype option on import or set low_memory=False.
  records=pd.read_csv(path_data_gener+path)
<ipython-input-1-3ce846fd6354>:29: DtypeWarning: Columns (60,61,62,63) have mixed types. Specify dtype option on import or set low_memory=False.
  records=pd.read_csv(path_data_gener+path)
<ipython-input-1-3ce846fd6354>:29: DtypeWarning: Columns (64,65) have mixed types. Specify dtype option on import or set low_memory=False.
  records=pd.read_csv(path_data_gener+path)
<ipython-input-1-3ce846fd6354>:29: Dtype

In [ ]:
joined[["compiledRelease/planning/budget/description","value_million_dolar"]].size

46096

In [ ]:
records

,Open Contracting ID,compiledRelease/id,compiledRelease/tender/id,compiledRelease/tender/title,compiledRelease/tender/status,compiledRelease/tender/awardCriteria,compiledRelease/tender/awardCriteriaDetails,compiledRelease/tender/submissionMethod,compiledRelease/tender/techniques/hasElectronicAuction,compiledRelease/tender/bidOpening/date,...,compiledRelease/tender/procurementIntention/rationale,compiledRelease/secondStage/id,compiledRelease/tender/techniques/hasFrameworkAgreement,compiledRelease/tender/contractPeriod/startDate,compiledRelease/tender/contractPeriod/endDate,Fecha,exchange_rate,Year,Avg,value_million_dolar
0,ocds-03ad3f-326319-1,326319-lpn-sbe-128-17-contratacion-servicios-m...,326319-lpn-sbe-128-17-contratacion-servicios-m...,LPN SBE 128-17 CONTRATACION DE SERVICIOS DE MA...,complete,priceOnly,Por Total,electronicAuction,True,2018-01-23T09:00:00-04:00,...,NaN,NaN,NaN,NaN,NaN,2018,5746.190,2018.0,0.858031,2.712687
1,ocds-03ad3f-329862-1,329862-adquisicion-jeringas-desechables-difere...,329862-adquisicion-jeringas-desechables-difere...,ADQUISICION DE JERINGAS DESECHABLES DE DIFEREN...,complete,priceOnly,Por Item,electronicAuction,True,2017-05-16T10:00:00-04:00,...,NaN,NaN,NaN,NaN,NaN,2017,5610.545,2017.0,0.837573,17.099990
2,ocds-03ad3f-333612-1,333612-adquisicion-reactivos-inmunohistoquimic...,333612-adquisicion-reactivos-inmunohistoquimic...,ADQUISICIÓN DE REACTIVOS DE INMUNOHISTOQUIMICA...,complete,priceOnly,Por Item,inPerson,NaN,2017-10-25T10:30:00-04:00,...,NaN,NaN,NaN,NaN,NaN,2017,5610.545,2017.0,0.837573,9.685412
3,ocds-03ad3f-335119-1,335119-adquisicion-set-bombas-infusion-volumet...,335119-adquisicion-set-bombas-infusion-volumet...,ADQUISICION DE SET DE BOMBAS DE INFUSION VOLUM...,complete,priceOnly,Por Item,electronicAuction,True,2017-11-23T10:00:00-04:00,...,NaN,NaN,NaN,NaN,NaN,2017,5610.545,2017.0,0.837573,38.304046
4,ocds-03ad3f-341654-1,341654-contratacion-servicio-mantenimiento-rep...,341654-contratacion-servicio-mantenimiento-rep...,CONTRATACIÓN DE SERVICIO DE MANTENIMIENTO Y RE...,complete,priceOnly,Por Lote,inPerson,NaN,2018-05-07T10:00:00-04:00,...,NaN,NaN,NaN,NaN,NaN,2018,5746.190,2018.0,0.858031,6.427474
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23043,ocds-03ad3f-385393-1,385393-construccion-bloques-sanitarios-escuela...,385393-construccion-bloques-sanitarios-escuela...,CONSTRUCCION DE BLOQUES DE SANITARIOS EN LA ES...,complete,priceOnly,Por Lote,inPerson,NaN,2020-12-01T07:40:00-04:00,...,NaN,NaN,NaN,NaN,NaN,2020,6742.370,2020.0,0.884355,0.269998
23044,ocds-03ad3f-385395-1,385395-construccion-sede-direccion-departament...,385395-construccion-sede-direccion-departament...,CONSTRUCCION DE LA SEDE DE LA DIRECCION DEPART...,complete,priceOnly,Por Total,inPerson,NaN,2020-10-15T09:30:00-04:00,...,NaN,NaN,NaN,NaN,NaN,2020,6742.370,2020.0,0.884355,2.179626
23045,ocds-03ad3f-385554-1,385554-adquisicion-acondicionadores-aire-1-167...,385554-adquisicion-acondicionadores-aire-1,Adquisición de acondicionadores de aire,complete,priceOnly,Por Total,inPerson,NaN,2020-11-19T08:40:00-04:00,...,NaN,NaN,NaN,NaN,NaN,2020,6742.370,2020.0,0.884355,0.058112
23046,ocds-03ad3f-385640-1,385640-techado-cancha-construccion-escenario-g...,385640-techado-cancha-construccion-escenario-g...,Techado para cancha y construcción de escenari...,complete,priceOnly,Por Total,inPerson,NaN,2020-11-16T08:30:00-04:00,...,NaN,NaN,NaN,NaN,NaN,2020,6742.370,2020.0,0.884355,0.880134


In [ ]:
CPI=CPI[["Year","Avg"]]